In [5]:
pip install flask tensorflow flask_cors

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
import tensorflow.keras.models as keras
import joblib

In [9]:
df = pd.read_csv('Final_Augmented_dataset_Diseases_and_Symptoms.csv')
print(df.columns)

Index(['diseases', 'anxiety and nervousness', 'depression',
       'shortness of breath', 'depressive or psychotic symptoms',
       'sharp chest pain', 'dizziness', 'insomnia',
       'abnormal involuntary movements', 'chest tightness',
       ...
       'stuttering or stammering', 'problems with orgasm', 'nose deformity',
       'lump over jaw', 'sore in nose', 'hip weakness', 'back swelling',
       'ankle stiffness or tightness', 'ankle weakness', 'neck weakness'],
      dtype='object', length=378)


In [11]:
X = df.drop('diseases', axis=1)
X = X.fillna(0)

In [13]:
scaler = StandardScaler()
scaler.fit(X)

joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [15]:
app = Flask(__name__)
CORS(app)  # Enable CORS to allow communication between frontend and backend

model = keras.load_model('disease_prediction_nn_model.h5')
scaler = joblib.load('scaler.pkl')  # Load scaler if used during preprocessing

# List of possible diseases (ensure these match your model's output classes)
disease_classes = [
    "Disease 1", "Disease 2", "Disease 3", "Disease 4", "Disease 5",
    "Disease 6", "Disease 7", "Disease 8", "Disease 9", "Disease 10"
]

@app.route('/predict', methods=['POST'])
def predict_disease():
    try:
        # Get symptom data from POST request (JSON format)
        input_data = request.json['symptoms']  # Example: {"symptoms": [1, 0, 1, ...]}
        
        # Convert input data to NumPy array and reshape for prediction
        input_array = np.array(input_data).reshape(1, -1)

        # Scale input data (if scaler was used during training)
        input_scaled = scaler.transform(input_array)

        # Make prediction using the loaded model
        predictions = model.predict(input_scaled)

        # Get the predicted class (index of highest probability)
        predicted_class_index = np.argmax(predictions)

        # Get disease name corresponding to predicted class index
        predicted_disease = disease_classes[predicted_class_index]

        return jsonify({
            'predicted_disease': predicted_disease,
            'probabilities': predictions.tolist()
        })
    except Exception as e:
        return jsonify({'error': str(e)})

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "/home/gaggarharsh31/anaconda3/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/gaggarharsh31/anaconda3/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/gaggarharsh31/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/gaggarharsh31/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/gaggarharsh31/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 

SystemExit: 1

/home/gaggarharsh31/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
